In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [9]:
import os
from groq import Groq

client = Groq()
#filename = os.path.dirname(__file__) + "/Recording.mp3"
filename="Resources_sample-calls.mp3"
with open(filename, "rb") as file:
    transcription = client.audio.transcriptions.create(
      file=(filename, file.read()),
      model="whisper-large-v3",
      response_format="verbose_json",
    )
    print(transcription.text)
      

 This call is now being recorded. Parker Scarves, how may I help you? I brought a scarf online for my wife and it turns out they shipped the wrong color. Oh, I am so sorry, sir. I got it for her birthday, which is tonight, and now I'm not 100% sure what I need to do. Okay, let me see if I can help you. Do you have the item number of the Parker Scarves? I don't think so. It's called a New Yorker, I think. Excellent. Okay. What color did you want the New Yorker in? Blue. The one that shipped was light blue. I wanted the darker one. Did you want navy blue or royal blue? What's the difference there? The royal blue is a bit brighter. That's the one I want. Okay. What zip code are you located in? 19406. It appears that we do not I sorry that we do have that item in stock at Karen Boutique at the Hunter Mall Is that close by It is It right by my office Okay What is your name sir Charlie Johnson. Charlie Johnson? Is that J-O-H-N-S-O-N? Yes, ma'am. And, Mr. Johnson, do you have the Parker scarf

In [34]:
transcription=transcription.text

In [ ]:
from langchain.prompts import ChatPromptTemplate

output_schema="""
{
  "call_transcript": [
    "CSR: ",
    "Custumer: ",
  ]
}

"""

transcript_template = """
List each line of dialogue in the transcript provided between <transcript></transcript> tags, labeling the speaker appropriately.
<transcript>{transcription}</transcript>

Format your response using the following JSON schema:
{output_schema} 

"""

prompt= ChatPromptTemplate.from_template(
  transcript_template,
  partial_variables={
        "output_schema": output_schema,
        },
   
)


In [36]:
prompt

ChatPromptTemplate(input_variables=['transcription'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['transcription'], input_types={}, partial_variables={'output_schema': '\n{\n  "call_transcript": [\n    "CSR: ",\n    "Custumer: ",\n  ]\n}\n\n'}, template='\nList each line of dialogue in the transcript provided between <transcript></transcript> tags, labeling the speaker appropriately.\n<transcript>{transcription}</transcript>\n\nFormat your response using the following JSON schema:\n{output_schema} \n\n'), additional_kwargs={})])

In [42]:
from langchain_groq import ChatGroq

llm= ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    timeout=None,
    max_retries=2,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)
    

c:\Users\utente\OneDrive\Bureau\project\project\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! stream is not default parameter.
                    stream was transferred to model_kwargs.
                    Please confirm that stream is what you intended.
  warnings.warn(
c:\Users\utente\OneDrive\Bureau\project\project\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! response_format is not default parameter.
                    response_format was transferred to model_kwargs.
                    Please confirm that response_format is what you intended.
  warnings.warn(


In [45]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

chain= ( prompt
        |llm
        |parser
        )

response= chain.invoke(transcription)

In [51]:
type(response)

dict

In [48]:
import json
from pathlib import Path
from rich.console import Console
from rich.markdown import Markdown

console = Console()

pretty_print_json = lambda x: console.print(Markdown(f"```json\n{json.dumps(json.loads(x), indent=2)}\n```"))
response_json = json.dumps(response, indent=2)
pretty_print_json(response_json)


                                                                                                                   
 {                                                                                                                 
   "call_transcript": [                                                                                            
     "CSR: This call is now being recorded. Parker Scarves, how may I help you?",                                  
     "Customer: I brought a scarf online for my wife and it turns out they shipped the wrong color.",              
     "CSR: I am so sorry, sir.",                                                                                   
     "Customer: I got it for her birthday, which is tonight, and now I'm not 100% sure what I need to do.",        
     "CSR: Okay, let me see if I can help you.",                                                                   
     "Customer: Do you have the item number of the Parker Scarves?",                                               
     "CSR: I don't think so.",                                                                                     
     "Customer: It's called a New Yorker, I think.",                                                               
     "CSR: Excellent.",                                                                                            
     "Customer: What color did you want the New Yorker in?",                                                       
     "CSR: Blue.",                                                                                                 
     "Customer: The one that shipped was light blue.",                                                             
     "CSR: I wanted the darker one.",                                                                              
     "Customer: Did you want navy blue or royal blue?",                                                            
     "CSR: What's the difference there?",                                                                          
     "Customer: The royal blue is a bit brighter.",                                                                
     "CSR: That's the one I want.",                                                                                
     "Customer: What zip code are you located in?",                                                                
     "CSR: 19406.",                                                                                                
     "CSR: It appears that we do not I sorry that we do not have that item in stock at Karen Boutique at the Hunte 
 Mall.",                                                                                                           
     "Customer: Is that close by?",                                                                                
     "CSR: It is.",                                                                                                
     "Customer: It right by my office.",                                                                           
     "CSR: Okay, what is your name, sir?",                                                                         
     "Customer: Charlie Johnson.",                                                                                 
     "CSR: Charlie Johnson? Is that J-O-H-N-S-O-N?",                                                               
     "Customer: Yes, ma'am.",                                                                                      
     "CSR: And, Mr. Johnson, do you have the Parker scarf in light blue with you now?",                            
     "Customer: I do.",                                                                                            
     "CSR: They shipped it to my office.",                                                                         
     "Customer: It just came in not that long ago.",    

In [49]:
output_directory = "data"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

file_path = os.path.join(output_directory, f"Call transcript result.json")

In [50]:
with open(file_path, 'w') as file:
    file.write(response_json)

print(f"Transcript  written to local file: {file_path}")

Transcript  written to local file: data\Call transcript result.json
